In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings('ignore')

In [16]:
X_train=pd.read_csv("/kaggle/input/citadel/X_train.csv")
y_train=pd.read_csv("/kaggle/input/citadel/y_train.csv")
X_train=X_train.drop(['Unnamed: 0'],axis=1)
y_train=y_train.drop(['Unnamed: 0'],axis=1)
X_test=pd.read_csv("/kaggle/input/citadel/X_test.csv")
y_test=pd.read_csv("/kaggle/input/citadel/y_test.csv")
X_test=X_test.drop(['Unnamed: 0'],axis=1)
y_test=y_test.drop(['Unnamed: 0'],axis=1)

In [5]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [6]:
xgdmat=xgb.DMatrix(X_train,y_train)
our_params_xgb={'learning_rate':0.07,'seed':0,'subsample':0.9,'colsample_bytree':0.7,'gamma':6,
           'objective':'binary:logistic','max_depth':18,'min_child_weight':4,'alpha':0.001,'lambda':0.01,'n_jobs':-1}
final_xgb = xgb.train(our_params_xgb,xgdmat,num_boost_round=209)

In [7]:
testdmat=xgb.DMatrix(X_test,y_test)
y_pred_xgb=final_xgb.predict(testdmat)
y_pred_xgb[y_pred_xgb>0.5]=1
y_pred_xgb[y_pred_xgb<=0.5]=0
y_pred_xgb

In [8]:
import shap
shap.initjs()

In [23]:
X_test.shape

In [24]:
X_shap = X_test.head(1929)

In [26]:
y_shap = y_test.head(1929)

In [30]:
shap_values_xgb=shap.TreeExplainer(final_xgb).shap_values(X_shap) 

In [34]:
import matplotlib.pyplot as plt
%matplotlib inline

shap.summary_plot(shap_values_xgb,X_shap,show=False)
plt.savefig('shapsummary.jpg',format='jpg',bbox_inches='tight',pad_inches=2,dpi=300)

In [35]:
explainer_xgb=shap.TreeExplainer(final_xgb)

In [37]:
y_proba_xgb=final_xgb.predict(testdmat)

In [42]:
y_proba_shap = y_proba_xgb[:1929]

In [44]:
y_proba_shap_sorted=sorted(y_proba_shap,reverse=True)
y_proba_shap_sorted[0]

In [45]:
for i in range(len(y_proba_shap)):
    if y_proba_shap[i]>=y_proba_shap_sorted[1]:
        print(i)

In [52]:
shap.force_plot(explainer_xgb.expected_value,shap_values_xgb[237,:],X_shap.iloc[237,:],link="logit",show=False)

In [49]:
for i in range(len(y_proba_shap)):
    if y_proba_shap[i]<=y_proba_shap_sorted[-1]:
        print(i)

In [53]:
shap.force_plot(explainer_xgb.expected_value,shap_values_xgb[16,:],X_shap.iloc[16,:],link="logit",show=False)